In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder 
import warnings
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from catboost import CatBoostClassifier

pd.set_option('max_columns', None)

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')

In [ ]:
corr = train.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
num_param = ['cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont9','cont10']
categorical_important_param = ['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat10',
                              'cat11','cat12','cat13','cat14','cat15','cat16','cat17','cat18']
total_param = num_param+categorical_important_param

In [ ]:
X = train[total_param]
y = train['target']

In [ ]:
encoder = ce.CatBoostEncoder(cols=categorical_important_param)
encoder.fit(X,y)
X_cleaned = encoder.transform(X)

In [ ]:
min_max_scalar = StandardScaler().fit(X_cleaned)
X_cleaned = min_max_scalar.transform(X_cleaned)

In [ ]:
model = CatBoostClassifier(iterations=2600,
                             learning_rate=0.002,
                             depth=12,
                             random_seed = 42,
                             bagging_temperature = 0.4,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20,loss_function = 'Logloss',
                             eval_metric = 'AUC')
model.fit(X_cleaned,y)

In [ ]:
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
X_test = test[total_param]
X_cleaned_test = encoder.transform(X_test)
X_cleaned_test = min_max_scalar.transform(X_cleaned_test)
y_pred = model.predict_proba(X_cleaned_test)[:,1]
X_d = test.id.to_list()
import random
d = pd.DataFrame()
for i in range(len(X_d)):
    dd = {'id':int(X_d[i]), 'target':(y_pred[i])}
    d = d.append(dd, ignore_index=True)

d["id"] = d["id"].astype(int)
d.to_csv('sub4.csv',index = False)